### **Self-Supervised Pretraining on Images**
 **Goal:** Use a self-supervised learning task (image rotation prediction) to pretrain a model without labels, then fine-tune it for image classification.

**Load and Preprocess the MNIST Dataset**

In [47]:
!pip install tensorflow numpy matplotlib scikit-learn

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import random

In [48]:
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()

# Normalize to [0, 1] and reshape
x_train = x_train.astype('float32') / 255.
x_train = np.expand_dims(x_train, -1)  # shape: (num_samples, 28, 28, 1)
print("Shape:", x_train.shape)

Shape: (60000, 28, 28, 1)


**Create Rotated Images and Labels**

In [49]:
def rotate_images(images):
    rotated = []
    labels = []
    for img in images:
        angle = np.random.choice([0, 90, 180, 270])
        if angle == 0:
            new_img = img
        elif angle == 90:
            new_img = tf.image.rot90(img, k=1).numpy()
        elif angle == 180:
            new_img = tf.image.rot90(img, k=2).numpy()
        else:
            new_img = tf.image.rot90(img, k=3).numpy()
        rotated.append(new_img)
        labels.append(angle // 90)  # 0:0°, 1:90°, 2:180°, 3:270°
    return np.array(rotated), np.array(labels)

x_ssl, y_ssl = rotate_images(x_train[:10000])
print("SSL images shape:", x_ssl.shape)
print("Rotation labels:", np.unique(y_ssl, return_counts=True))

SSL images shape: (10000, 28, 28, 1)
Rotation labels: (array([0, 1, 2, 3]), array([2518, 2497, 2511, 2474]))


**Build the Rotation Prediction Model**

In [50]:
def build_ssl_model():
    model = models.Sequential([
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(2),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='softmax')  # 4 classes for rotation
    ])
    return model

ssl_model = build_ssl_model()
ssl_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ssl_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 224,260 (876.02 KB)

 Trainable params: 224,260 (876.02 KB)

 Non-trainable params: 0 (0.00 B)

**Train the SSL Model**

In [51]:
ssl_model.fit(x_ssl, y_ssl, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.7461 - loss: 0.6742 - val_accuracy: 0.9290 - val_loss: 0.2276
Epoch 2/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 66ms/step - accuracy: 0.9488 - loss: 0.1484 - val_accuracy: 0.9660 - val_loss: 0.1055
Epoch 3/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.9729 - loss: 0.0763 - val_accuracy: 0.9640 - val_loss: 0.1083
Epoch 4/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - accuracy: 0.9764 - loss: 0.0653 - val_accuracy: 0.9720 - val_loss: 0.0760
Epoch 5/5
141/141 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.9840 - loss: 0.0473 - val_accuracy: 0.9290 - val_loss: 0.2144


### **Summary**

*   **Goal:** Learn useful image features by training a model to predict image rotations (0°, 90°, 180°, 270°) without using class labels.
*   **Dataset:** MNIST handwritten digits (only input images were used—no labels).
*   **Self-Supervised Task:** Rotated each image randomly and trained the model to classify the rotation angle.
*   **Model Architecture:** A convolutional neural network (CNN) with:
Two Conv2D layers, Two MaxPooling2D layers, Dense layers ending in 4 outputs (for the 4 rotation classes)
*   **Training:** Trained on 10,000 rotated images for 5 epochs using categorical crossentropy.
*   **Feature Extraction**: Used the trained model as a feature encoder (excluding final softmax layer).

**Why Self-Supervised?** No labels from the original dataset were used; the model created its own "pseudo-labels" from image rotations.